In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EMOTIONS=['anger','contempt','disgust','fear',
          'happy','neutral','sad','surprise']

NUM_CLASSES=8
BATCH_SIZE=32
EPOCHS=25
IMG_SIZE=224


In [5]:
class AffectNetDataset(Dataset):

    def __init__(self,root,split,transform=None):
        self.transform=transform
        self.images=[]
        self.labels=[]

        root=Path(root)/split

        for i,e in enumerate(EMOTIONS):
            p=root/e
            if p.exists():
                for img in p.glob("*.*"):
                    self.images.append(img)
                    self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        img=Image.open(self.images[idx]).convert("RGB")
        if self.transform:
            img=self.transform(img)
        return img,self.labels[idx]


In [6]:
train_tf=transforms.Compose([
 transforms.Resize((IMG_SIZE,IMG_SIZE)),
 transforms.RandomHorizontalFlip(),
 transforms.RandomRotation(15),
 transforms.ColorJitter(0.3,0.3,0.3,0.1),
 transforms.RandomResizedCrop(224,scale=(0.8,1)),
 transforms.ToTensor(),
 transforms.Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
])

test_tf=transforms.Compose([
 transforms.Resize((IMG_SIZE,IMG_SIZE)),
 transforms.ToTensor(),
 transforms.Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
])


In [7]:
data="/kaggle/input/datasets/bouyassine11/affectnetcoorect/archive (3)"

train_ds=AffectNetDataset(data,"Train",train_tf)
test_ds=AffectNetDataset(data,"Test",test_tf)

train_loader=DataLoader(train_ds,BATCH_SIZE,True,num_workers=2)
test_loader=DataLoader(test_ds,BATCH_SIZE,False,num_workers=2)

In [8]:
cw=compute_class_weight("balanced",
                        classes=np.unique(train_ds.labels),
                        y=train_ds.labels)

cw=torch.tensor(cw,dtype=torch.float).to(device)

criterion=nn.CrossEntropyLoss(weight=cw)

In [9]:
def get_resnet():

 m=models.resnet50(pretrained=True)

 for p in m.parameters():
  p.requires_grad=False

 for p in m.layer4.parameters():
  p.requires_grad=True

 in_f=m.fc.in_features
 m.fc=nn.Sequential(
  nn.Linear(in_f,512),
  nn.ReLU(),
  nn.Dropout(0.5),
  nn.Linear(512,NUM_CLASSES)
 )

 return m.to(device)

In [10]:
def get_efficient():

 m=models.efficientnet_v2_s(pretrained=True)

 for p in m.parameters():
  p.requires_grad=False

 for p in m.features[-1].parameters():
  p.requires_grad=True

 in_f=m.classifier[1].in_features

 m.classifier=nn.Sequential(
  nn.Linear(in_f,512),
  nn.ReLU(),
  nn.Dropout(0.5),
  nn.Linear(512,NUM_CLASSES)
 )

 return m.to(device)

In [12]:
def get_vit():

 m=models.vit_b_16(pretrained=True)

 for p in m.parameters():
  p.requires_grad=False

 for p in m.encoder.layers[-1].parameters():
  p.requires_grad=True

 in_f=m.heads.head.in_features

 m.heads.head=nn.Sequential(
  nn.Linear(in_f,512),
  nn.ReLU(),
  nn.Dropout(0.6),
  nn.Linear(512,NUM_CLASSES)
 )

 return m.to(device)

In [13]:
def train_model(model,name):

 opt=optim.Adam([
  {'params':filter(lambda p:p.requires_grad,
                   model.parameters()),'lr':1e-4}
 ],weight_decay=1e-4)

 sch=optim.lr_scheduler.ReduceLROnPlateau(
  opt,'min',patience=3,factor=0.5)

 tl,vl,ta,va=[],[],[],[]

 for e in range(EPOCHS):

  model.train()
  loss_sum,cor,tot=0,0,0

  for x,y in tqdm(train_loader):
   x,y=x.to(device),y.to(device)
   opt.zero_grad()
   out=model(x)
   loss=criterion(out,y)
   loss.backward()
   opt.step()

   loss_sum+=loss.item()
   cor+=(out.argmax(1)==y).sum().item()
   tot+=y.size(0)

  ta.append(100*cor/tot)
  tl.append(loss_sum/len(train_loader))

  model.eval()
  loss_sum,cor,tot=0,0,0
  preds,labels=[],[]

  with torch.no_grad():
   for x,y in test_loader:
    x,y=x.to(device),y.to(device)
    out=model(x)
    loss=criterion(out,y)

    loss_sum+=loss.item()
    p=out.argmax(1)

    cor+=(p==y).sum().item()
    tot+=y.size(0)

    preds+=p.cpu().tolist()
    labels+=y.cpu().tolist()

  va.append(100*cor/tot)
  vl.append(loss_sum/len(test_loader))

  print(name,e+1,ta[-1],va[-1])

  sch.step(vl[-1])

 return {"tl":tl,"vl":vl,
         "ta":ta,"va":va,
         "p":preds,"l":labels}

In [ ]:
results={}

results["ResNet50"]=train_model(get_resnet(),"ResNet50")
results["EfficientNetV2"]=train_model(get_efficient(),"EfficientNetV2")
results["ViT"]=train_model(get_vit(),"ViT")
